In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv

In [48]:
image_list = []
label_list = []
for i in range(1,901):
    name = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/pencilCase/test'
    name = name + str(i) + '.jpg'
    image_list.append(name)
    label_list.append([1,0,0])
for i in range(1,901):
    name = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/tissue/tissue'
    name = name + str(i) + '.jpg'
    image_list.append(name)
    label_list.append([0,1,0])
for i in range(1,901):
    name = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/watch/watch'
    name = name + str(i) + '.jpg'
    image_list.append(name)
    label_list.append([0,0,1])    

In [49]:
def _read_list_image(input):
    return [cv.imread(input[x], cv.IMREAD_GRAYSCALE) for x in range(2700)]

In [98]:
img = np.stack(_read_list_image(image_list))
label = np.array(label_list, dtype=np.float32)#label float화

In [99]:
img = tf.reshape(img, shape=[-1,2304])
img = tf.to_float(img,name='ToFloat')#img float화

In [100]:
dataset = tf.data.Dataset.from_tensor_slices((img,label))
dataset = dataset.repeat()
dataset = dataset.shuffle(5000)
dataset = dataset.batch(20)

In [101]:
iterator = dataset.make_initializable_iterator()
image_stack, label_stack = iterator.get_next()
next_element = iterator.get_next()

In [54]:
sess = tf.Session()
sess.run(iterator.initializer)
sess.run(tf.global_variables_initializer())

In [84]:
x, y = sess.run(next_element)#아래의 placeholder가 씹힘 주의할 것

In [95]:
x = tf.placeholder(dtype=tf.float32, shape = )

[ 81.  90.  82. ... 117. 117. 117.]


In [102]:
x = tf.placeholder(dtype=tf.float32, shape = [None, 2304])# 48 * 48 size image
y = tf.placeholder(dtype=tf.float32, shape = [None, 3])# pencilcase, tissue, watch

In [103]:
x_image = tf.reshape(x, [-1,48,48,1]) #벡터화된 이미지 값을 매트릭스화

In [104]:
def weight_variable(shape): #weight 정의 함수. kernel, model등에 쓰임
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

In [105]:
def bias_variable(shape): #bias 정의 함수, 위와 마찬가지로 쓰임
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [106]:
def convolution(x, kernel): #합성곱을 한칸씩 이동해서 하며 이를 통해 만들어진 feature map의 크기가 같음
    return tf.nn.conv2d(x, kernel, strides=[1,1,1,1], padding='SAME') #padding='SAME' 때문에

In [107]:
def pooling(x): # 2x2 를 1x1로 maxpooling 하는 함수
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [108]:
kernel_1 = weight_variable([5,5,1,32]) # 1개의 input에 대해서 5x5의 커널을 32개의 채널로 만듬
b_conv_1 = bias_variable([32]) #커널의 채널수에 맞는 bias 만듬

In [109]:
conv_1 = tf.nn.relu(convolution(x_image, kernel_1) + b_conv_1) #합성곱 후 활성함수에 넣은 activation map
pool_1 = pooling(conv_1) #activation map maxpooling으로 간소화 48x48 -> 24x24

In [110]:
kernel_2 = weight_variable([5,5,32,64]) # 32개의 input에 대해서 5x5의 커널을 64개의 채널로 만듬
b_conv_2 = bias_variable([64]) #커널의 채널수에 맞는 bias

In [111]:
conv_2 = tf.nn.relu(convolution(pool_1, kernel_2) + b_conv_2)
pool_2 = pooling(conv_2) # 위 행위 반복 24x24 -> 12x12

In [112]:
kernel_3 = weight_variable([5,5,64,128])
b_conv_3 = bias_variable([128])

In [113]:
conv_3 = tf.nn.relu(convolution(pool_2, kernel_3) + b_conv_3)
pool_3 = pooling(conv_3)# 12x12 -> 6x6

In [114]:
w_fc1 = weight_variable([6*6*128, 1024])
b_fc1 = bias_variable([1024])
pool_flat = tf.reshape(pool_3, [-1, 6*6*128])

In [115]:
h_fc1 = tf.nn.relu(tf.matmul(pool_flat, w_fc1) + b_fc1)
keep_prob = tf.placeholder("float")
h_fc1_dropout = tf.nn.dropout(h_fc1, keep_prob)

In [116]:
w_fc2 = weight_variable([1024,3])
b_fc2 = bias_variable([3])

In [117]:
y_conv = tf.nn.softmax(tf.matmul(h_fc1_dropout, w_fc2) + b_fc2) #softmax까지 써서 최종적인 판단 결과 도출, 이놈이 hypothesis

In [118]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(prediction, "float"))

In [119]:
sess.run(tf.global_variables_initializer())
sess.run(iterator.initializer)#반드시 변수 initializer해줘야

In [123]:
for i in range(1001):
    x_data, y_data = sess.run(next_element)    
    if i%25 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:x_data, y:y_data, keep_prob:1.0})
        print("step ",i," : ",train_accuracy)
    sess.run(train_step, feed_dict={x:x_data, y:y_data, keep_prob: 0.5})

step  0  :  0.4
step  25  :  0.5
step  50  :  0.3
step  75  :  0.1
step  100  :  0.25
step  125  :  0.4
step  150  :  0.25
step  175  :  0.4
step  200  :  0.1
step  225  :  0.2
step  250  :  0.55
step  275  :  0.4
step  300  :  0.3
step  325  :  0.25
step  350  :  0.15
step  375  :  0.25
step  400  :  0.5
step  425  :  0.5
step  450  :  0.2
step  475  :  0.3
step  500  :  0.45
step  525  :  0.5
step  550  :  0.45
step  575  :  0.3
step  600  :  0.35
step  625  :  0.2
step  650  :  0.35
step  675  :  0.35
step  700  :  0.6
step  725  :  0.4
step  750  :  0.25
step  775  :  0.35
step  800  :  0.45
step  825  :  0.4
step  850  :  0.45
step  875  :  0.3
step  900  :  0.15
step  925  :  0.35
step  950  :  0.45
step  975  :  0.4
step  1000  :  0.2


In [174]:
#path = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/pencilCase/test1500.jpg'
#path = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/tissue/tissue910.jpg'
path = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/watch/watch905.jpg'

test_x = cv.imread(path, cv.IMREAD_GRAYSCALE)
test_x = tf.reshape(test_x, shape=[-1,2304])
test_x = tf.to_float(test_x,name='ToFloat')
test_x = sess.run(test_x)#tensor object는 sess.run하면 본래의 자료형이 튀어나옴
print(test_x[0])

[19. 20. 21. ...  3.  3. 11.]


In [175]:
test_y1 = [[1,0,0]]
test_y1 = np.array(test_y1, dtype=np.float32)
pred = sess.run(accuracy, feed_dict={x:test_x, y:test_y1, keep_prob:1.0})
print(test_y1[0])
print(pred)

test_y2 = [[0,1,0]]
test_y2 = np.array(test_y2, dtype=np.float32)
pred = sess.run(accuracy, feed_dict={x:test_x, y:test_y2, keep_prob:1.0})
print(test_y2[0])
print(pred)

test_y3 = [[0,0,1]]
test_y3 = np.array(test_y3, dtype=np.float32)
pred = sess.run(accuracy, feed_dict={x:test_x, y:test_y3, keep_prob:1.0})
print(test_y3[0])
print(pred)

[1. 0. 0.]
1.0
[0. 1. 0.]
0.0
[0. 0. 1.]
0.0
